In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1140.86,0.04,-0.08,-0.01,0.03,0.00,-0.00,-0.02,0.01,-0.02,...,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.0,-0.00,-0.00
3,0.04,1504.40,-0.00,-0.00,0.05,0.02,0.01,0.02,0.01,0.01,...,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.0,-0.00,0.00
4,-0.08,-0.00,1413.84,-0.01,-0.03,0.00,0.03,-0.00,0.00,0.01,...,-0.01,0.00,0.00,-0.00,0.00,-0.00,0.00,0.0,0.00,-0.00
5,-0.01,-0.00,-0.01,757.77,-0.02,0.01,-0.02,-0.01,-0.01,-0.01,...,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00
6,0.03,0.05,-0.03,-0.02,513.25,-0.00,-0.00,-0.01,-0.01,-0.01,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,-0.00,-0.00
7,0.00,0.02,0.00,0.01,-0.00,311.44,-0.00,0.00,0.01,0.00,...,0.00,0.01,-0.00,0.00,-0.00,0.00,0.00,0.0,-0.00,-0.00
8,-0.00,0.01,0.03,-0.02,-0.00,-0.00,241.38,-0.00,0.00,0.01,...,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.0,-0.00,-0.00
9,-0.02,0.02,-0.00,-0.01,-0.01,0.00,-0.00,213.94,-0.00,0.01,...,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,0.00,-0.00
10,0.01,0.01,0.00,-0.01,-0.01,0.01,0.00,-0.00,157.11,-0.00,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.0,-0.00,0.00
11,-0.02,0.01,0.01,-0.01,-0.01,0.00,0.01,0.01,-0.00,140.33,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,-0.00,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00003,-0.00006,-0.00001,0.00004,0.00001,-0.00000,-0.00004,0.00001,-0.00005,...,-0.00002,-0.00001,-0.00007,0.00007,0.00003,0.00000,-0.00001,-0.00006,-0.00002,-0.00001
3,0.00003,1.00000,-0.00000,-0.00000,0.00006,0.00004,0.00001,0.00004,0.00003,0.00002,...,-0.00004,-0.00000,-0.00003,0.00000,0.00005,0.00001,-0.00001,0.00001,-0.00000,0.00003
4,-0.00006,-0.00000,1.00000,-0.00001,-0.00004,0.00000,0.00006,-0.00000,0.00001,0.00003,...,-0.00009,0.00002,0.00006,-0.00002,0.00000,-0.00007,0.00003,0.00006,0.00002,-0.00001
5,-0.00001,-0.00000,-0.00001,1.00000,-0.00004,0.00002,-0.00004,-0.00001,-0.00003,-0.00004,...,0.00003,0.00002,0.00003,0.00000,-0.00002,-0.00002,-0.00001,-0.00002,0.00010,0.00001
6,0.00004,0.00006,-0.00004,-0.00004,1.00000,-0.00001,-0.00001,-0.00002,-0.00003,-0.00002,...,-0.00002,0.00002,0.00004,-0.00004,0.00003,-0.00000,0.00003,-0.00002,-0.00002,-0.00002
7,0.00001,0.00004,0.00000,0.00002,-0.00001,1.00000,-0.00001,0.00001,0.00004,0.00002,...,0.00001,0.00006,-0.00001,0.00005,-0.00005,0.00003,0.00001,0.00003,-0.00003,-0.00001
8,-0.00000,0.00001,0.00006,-0.00004,-0.00001,-0.00001,1.00000,-0.00002,0.00002,0.00004,...,-0.00001,-0.00004,0.00000,0.00000,-0.00001,-0.00002,0.00003,-0.00002,-0.00002,-0.00005
9,-0.00004,0.00004,-0.00000,-0.00001,-0.00002,0.00001,-0.00002,1.00000,-0.00001,0.00004,...,0.00003,-0.00000,-0.00004,-0.00001,-0.00001,0.00001,-0.00001,0.00006,0.00002,-0.00001
10,0.00001,0.00003,0.00001,-0.00003,-0.00003,0.00004,0.00002,-0.00001,1.00000,-0.00003,...,0.00002,-0.00000,0.00003,-0.00006,-0.00001,0.00006,0.00002,-0.00003,-0.00005,0.00003
11,-0.00005,0.00002,0.00003,-0.00004,-0.00002,0.00002,0.00004,0.00004,-0.00003,1.00000,...,0.00001,-0.00002,-0.00002,0.00004,-0.00001,0.00003,-0.00002,0.00001,-0.00002,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.20982220474085025

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.13275350e+04 6.21871364e+04 4.05085615e+04 1.78863612e+04
 8.19763359e+03 3.02542197e+03 1.82046473e+03 1.42047897e+03
 9.37439232e+02 7.70002900e+02 6.64302436e+02 5.79251813e+02
 2.42996373e+02 1.07457177e+02 6.72530492e+01 6.02802745e+01
 3.99366086e+01 2.60960166e+01 1.43589243e+01 1.18280451e+01
 9.44562102e+00 7.46964889e+00 3.10089225e+00 1.43856978e+00
 4.72427059e-01 3.21836437e-01 1.07820825e-02 5.72201935e-03
 5.17593572e-04 1.83797422e-04 4.67708894e-06 1.40666127e-26]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.042089,0.973373,-0.224448,-0.015589,-0.008820,-0.004944,-0.003778,-0.003309,-0.002411,-0.002153,...,-0.000159,-0.000330,-0.000063,-0.000054,-0.000007,-0.000009,-0.000002,-0.000001,-2.564153e-07,-4.079575e-08
1,-0.089299,0.219320,0.970944,-0.027378,-0.015098,-0.008310,-0.006281,-0.005552,-0.004029,-0.003581,...,-0.000271,-0.000545,-0.000098,-0.000091,-0.000012,-0.000017,-0.000003,-0.000002,-3.285642e-07,-1.884994e-07
2,0.993945,0.059468,0.076129,-0.044370,-0.020580,-0.010714,-0.008036,-0.007070,-0.005083,-0.004546,...,-0.000332,-0.000686,-0.000130,-0.000109,-0.000015,-0.000020,-0.000005,-0.000004,-5.647435e-07,-2.603298e-07
3,0.039692,0.022873,0.025443,0.997137,-0.045898,-0.017663,-0.012664,-0.010932,-0.007717,-0.006839,...,-0.000483,-0.001008,-0.000185,-0.000166,-0.000023,-0.000030,-0.000007,-0.000005,-2.590221e-07,-3.002572e-07
4,0.019643,0.013486,0.014684,0.042879,0.997274,-0.035844,-0.022281,-0.018583,-0.012404,-0.010838,...,-0.000740,-0.001534,-0.000280,-0.000255,-0.000033,-0.000044,-0.000010,-0.000007,-1.194543e-06,-5.998302e-07
5,0.009658,0.007077,0.007596,0.016118,0.030974,0.994637,-0.069577,-0.045556,-0.023206,-0.019361,...,-0.001130,-0.002346,-0.000435,-0.000381,-0.000054,-0.000066,-0.000015,-0.000010,-1.849912e-06,-8.661271e-07
6,0.006829,0.005082,0.005401,0.010941,0.018492,0.057325,0.983777,-0.149757,-0.036829,-0.028410,...,-0.001376,-0.002884,-0.000525,-0.000467,-0.000064,-0.000083,-0.000018,-0.000013,-2.211468e-06,-1.281607e-06
7,0.007233,0.005375,0.005751,0.011377,0.018626,0.046755,0.135025,0.979634,-0.064582,-0.045735,...,-0.001869,-0.003940,-0.000724,-0.000640,-0.000087,-0.000111,-0.000026,-0.000016,-2.584198e-06,-1.511024e-06
8,0.004735,0.003527,0.003798,0.007338,0.011548,0.024197,0.043590,0.067713,-0.189427,-0.078911,...,-0.001895,-0.003966,-0.000724,-0.000650,-0.000088,-0.000114,-0.000025,-0.000017,-2.580780e-06,-1.017154e-06
9,0.004324,0.003244,0.003473,0.006690,0.010410,0.020599,0.033950,0.047413,0.938494,-0.163139,...,-0.002109,-0.004469,-0.000803,-0.000730,-0.000099,-0.000122,-0.000028,-0.000020,-3.552216e-06,-1.291951e-06


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0420887917803305,
 0.7806803423960937,
 0.923870943566419,
 0.002863001417256039,
 0.0027256798295161033,
 0.005362736065443241,
 0.01622331946050759,
 0.020365961779387765,
 1.1894270069036608,
 1.1631392213413552,
 1.0237752769780082,
 1.0417703386102999,
 1.036736121529304,
 1.0990365838041967,
 1.0274770898849424,
 1.087861791364248,
 1.0400721132332165,
 1.0886514483773155,
 1.0900682041517031,
 0.9923465523781889,
 0.9977385980642696,
 1.0860567270635217,
 0.031505015580021056,
 0.9581933597417014,
 0.11135184520264885,
 0.11596763356106543,
 1.3290100861302505,
 0.6784246934283142,
 0.06742353796531642,
 0.07082683959282421,
 0.10658616302736024,
 0.1052222710427676]